# SportsBar Product Dimension Processing Pipeline
## Medallion Architecture: Bronze → Silver → Gold

### 📌 NOTEBOOK OVERVIEW

This notebook processes SportsBar product data through the medallion architecture pattern, applying data quality transformations and business rule standardizations to create a production-ready product dimension table.

**Pipeline Purpose:** Ingest raw product data, apply quality checks and standardizations, then merge with parent company master data for unified analytics.

**Output:** `fmcg.gold.dim_products` - Unified product dimension table with division and category hierarchies

### 🔄 PROCESSING FLOW

| Layer | Table Name | Records | Purpose | Key Transformations |
|-------|-----------|---------|---------|---------------------|
| **Bronze** | `fmcg.bronze.products` | Raw count | Raw ingestion | Metadata tracking |
| **Silver** | `fmcg.silver.products` | Deduplicated | Quality & standardization | 7-step quality framework |
| **Gold (Staging)** | `fmcg.gold.sb_dim_products` | Business-ready | SportsBar product catalog | Select & rename |
| **Gold (Parent)** | `fmcg.gold.dim_products` | Merged | Master data integration | MERGE with parent company |

### 🎯 KEY BUSINESS LOGIC

**Product Classification Hierarchy:**
- Raw categories (Energy Bars, Protein Bars, etc.) map to divisions (Nutrition Bars, Breakfast Foods, etc.)
- Each product gets a deterministic `product_code` via SHA256 hash of product name
- Invalid product IDs (non-numeric) default to "999999" for joinability

**Data Quality Steps (7-Step Process):**
1. **Deduplication** - Remove duplicate product records by product_id
2. **Title Case Fix** - Standardize category names (energy bars → Energy Bars)
3. **Spelling Correction** - Fix typo: "Protien" → "Protein" throughout
4. **Division Mapping** - Map raw categories to business divisions
5. **Variant Extraction** - Extract variant info from product name (text in parentheses)
6. **Product Code Generation** - Create deterministic SHA256 hash for joins
7. **Product ID Validation** - Numeric validation with fallback for invalid IDs

### 💼 BUSINESS CONTEXT

The product dimension provides a curated view of SportsBar's product portfolio used for order fact table joins. It enriches raw sales data with product hierarchy information enabling analysis by division, category, and individual products. The product code serves as a conformed dimension key across the data warehouse.

## STEP 1: Import Required Libraries

All necessary PySpark and Delta Lake modules for the medallion transformation pipeline

In [0]:
# Import PySpark SQL functions for data transformations (withColumn, initcap, etc.)
from pyspark.sql import functions as F
# Import DeltaTable for MERGE operations (upsert patterns)
from delta.tables import DeltaTable

## STEP 2: Load Project Utilities & Initialize Notebook Widgets

Import centralized schema names from utilities notebook and configure pipeline parameters

In [0]:
# Load utilities from setup catalog - defines schema name constants (bronze_schema, silver_schema, gold_schema)
%run /Workspace/Project1/1_setup_catalog/utilities

In [0]:
# Verify schema names are loaded correctly (should print: bronze, silver, gold)
print(bronze_schema, silver_schema, gold_schema)

bronze silver gold


In [0]:
# Configure notebook widgets for parameterized execution (allows different catalogs/data sources)
dbutils.widgets.text("catalog", "fmcg", "Catalog")
dbutils.widgets.text("data_source", "products", "Data Source")

# Retrieve widget values
catalog = dbutils.widgets.get("catalog")
data_source = dbutils.widgets.get("data_source")

# Construct ADLS Gen2 path to products CSV files
# Pattern: abfss://container@storage.dfs.core.windows.net/data_source/*.csv
base_path = f"abfss://conatiner-de-practice@adlsgen2narayan.dfs.core.windows.net/{data_source}/*.csv"
# Alternative S3 path (commented out for Azure setup):
# base_path = f's3://sportsbar-final/{data_source}/*.csv'
print(base_path)

abfss://conatiner-de-practice@adlsgen2narayan.dfs.core.windows.net/products/*.csv


## STEP 3: BRONZE LAYER - Raw Data Ingestion

**Purpose:** Load raw product CSV files with full lineage tracking (timestamps, file names, file sizes)

**Medallion Pattern:** Data flows without any transformation - capture everything as-is

**Key Metadata Tracked:**
- `read_timestamp` - When the file was processed
- `file_name` - Source file identifier
- `file_size` - Raw data volume indicator

**Output Table:** `fmcg.bronze.products` (append mode for audit trail)

In [0]:
df = (
    spark.read.format("csv")                    # Read CSV format
        .option("header", True)                  # First row contains column names
        .option("inferSchema", True)             # Automatically detect column data types
        .load(base_path)                         # Load from ADLS path
        .withColumn("read_timestamp", F.current_timestamp())  # Add processing timestamp
        .select("*", "_metadata.file_name", "_metadata.file_size")  # Include file metadata
)

In [0]:
# Display data types and schema structure - verify correct parsing
df.printSchema()

root
 |-- product_name: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- read_timestamp: timestamp (nullable = false)
 |-- file_name: string (nullable = false)
 |-- file_size: long (nullable = false)



In [0]:
# Preview first 10 rows to visually inspect data quality and content
display(df.limit(10))

product_name,product_id,category,read_timestamp,file_name,file_size
SportsBar Energy Bar Choco Fudge (60g),25891101,energy bars,2025-11-30T13:39:48.677Z,products.csv,1388
SportsBar Energy Bar Choco Fudge (40g),25891102,energy bars,2025-11-30T13:39:48.677Z,products.csv,1388
SportsBar Energy Bar Choco Fudge (25g),25891103,energy bars,2025-11-30T13:39:48.677Z,products.csv,1388
SportsBar Protien Bar Peanut Crunch (45g),25891201,protien bars,2025-11-30T13:39:48.677Z,products.csv,1388
SportsBar Protien Bar Peanut Crunch (55g),25891202,protien bars,2025-11-30T13:39:48.677Z,products.csv,1388
SportsBar Protien Bar Peanut Crunch (65g),25891203,protien bars,2025-11-30T13:39:48.677Z,products.csv,1388
SportsBar Granola Crunch Honey Almond (400g),25891301,granola & cereals,2025-11-30T13:39:48.677Z,products.csv,1388
SportsBar Granola Crunch Honey Almond (300g),25891302,granola & cereals,2025-11-30T13:39:48.677Z,products.csv,1388
SportsBar Granola Crunch Honey Almond (200g),25891303,granola & cereals,2025-11-30T13:39:48.677Z,products.csv,1388
SportsBar Greek Yogurt Pro Vanilla (200g),25891401,recovery dairy,2025-11-30T13:39:48.677Z,products.csv,1388


In [0]:
# Write to bronze layer with Change Data Feed enabled for audit trail
df.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \  # Enable CDF for tracking all changes
 .mode("overwrite") \                            # Replace existing data
 .saveAsTable(f"{catalog}.{bronze_schema}.{data_source}")

## STEP 4: SILVER LAYER - Data Quality & Standardization

**Purpose:** Apply 7-step quality transformations to clean, standardize, and enrich product data

**Medallion Pattern:** Transform raw data into conformed, business-ready schema

**Quality Checks Applied:**
1. ✅ **Deduplication** - One record per product_id
2. ✅ **Title Case Standardization** - Category names consistent formatting
3. ✅ **Spelling Corrections** - Fix known typos (Protien → Protein)
4. ✅ **Business Rule Mapping** - Raw categories → division hierarchy
5. ✅ **Variant Extraction** - Parse variant info from product names
6. ✅ **Deterministic Key Generation** - SHA256 product_code for joins
7. ✅ **ID Validation** - Numeric checks with fallback for invalid IDs

**Output Table:** `fmcg.silver.products` (standardized, joinable dimension)

In [0]:
# Read from bronze layer - start with raw data
bronze_query = f"SELECT * FROM {catalog}.{bronze_schema}.{data_source};"
df_bronze = spark.sql(bronze_query)
df_bronze.show(10)

+--------------------+----------+-----------------+--------------------+------------+---------+
|        product_name|product_id|         category|      read_timestamp|   file_name|file_size|
+--------------------+----------+-----------------+--------------------+------------+---------+
|SportsBar Energy ...|  25891101|      energy bars|2025-11-30 13:40:...|products.csv|     1388|
|SportsBar Energy ...|  25891102|      energy bars|2025-11-30 13:40:...|products.csv|     1388|
|SportsBar Energy ...|  25891103|      energy bars|2025-11-30 13:40:...|products.csv|     1388|
|SportsBar Protien...|  25891201|     protien bars|2025-11-30 13:40:...|products.csv|     1388|
|SportsBar Protien...|  25891202|     protien bars|2025-11-30 13:40:...|products.csv|     1388|
|SportsBar Protien...|  25891203|     protien bars|2025-11-30 13:40:...|products.csv|     1388|
|SportsBar Granola...|  25891301|granola & cereals|2025-11-30 13:40:...|products.csv|     1388|
|SportsBar Granola...|  25891302|granola

### Silver Layer Transformations (7-Step Quality Framework)

#### QUALITY STEP 1️⃣: Drop Duplicates by Product ID

**Objective:** Ensure single record per product_id (business key)

**Method:** Drop duplicate rows keeping first occurrence

**Impact:** Eliminates redundant records, improves join efficiency

In [0]:
# Count rows before deduplication
print('Rows before duplicates dropped: ', df_bronze.count())
# Keep only unique product_id records (drop duplicates on this key)
df_silver = df_bronze.dropDuplicates(['product_id'])
# Report deduplication impact
print('Rows after duplicates dropped: ', df_silver.count())

Rows before duplicates dropped:  20
Rows after duplicates dropped:  18


#### QUALITY STEP 2️⃣: Title Case Fix for Category Names

**Objective:** Standardize category formatting (lowercase/mixed case → Title Case)

**Examples:** 
- energy bars → Energy Bars
- PROTEIN BARS → Protein Bars
- protien bars → Protien Bars (note: typo to be fixed next)

**Business Impact:** Consistent category naming for reporting and analysis

In [0]:
# Preview current category values to see inconsistent formatting
df_silver.select('category').distinct().show()

+-----------------+
|         category|
+-----------------+
|      energy bars|
|     protien bars|
|granola & cereals|
|   recovery dairy|
|   healthy snacks|
|  electrolyte mix|
+-----------------+



In [0]:
# Apply title case to category: capitalize first letter of each word
# NULL values preserved as NULL (safety check with when/otherwise)
df_silver = df_silver.withColumn(
    "category",
    F.when(F.col("category").isNull(), None)      # Preserve NULLs
     .otherwise(F.initcap("category"))             # Apply title case (capitalize first letters)
)

In [0]:
# Verify title case was applied - all categories should now start with capital letters
df_silver.select('category').distinct().show()

+-----------------+
|         category|
+-----------------+
|      Energy Bars|
|     Protien Bars|
|Granola & Cereals|
|   Recovery Dairy|
|   Healthy Snacks|
|  Electrolyte Mix|
+-----------------+



#### QUALITY STEP 3️⃣: Fix Spelling Mistake - "Protien" → "Protein"

**Objective:** Correct common misspelling throughout dataset

**Regex Pattern:** Case-insensitive replacement ((?i)Protien)

**Target Columns:** product_name and category

**Business Impact:** Ensures consistent product naming across all tables

In [0]:
# Replace 'protien' → 'protein' in both product_name and category
# Regex pattern (?i) makes the search case-insensitive (matches Protien, PROTIEN, protien, etc.)
df_silver = (
    df_silver
    .withColumn(
        "product_name",
        F.regexp_replace(F.col("product_name"), "(?i)Protien", "Protein")  # Fix in product names
    )
    .withColumn(
        "category",
        F.regexp_replace(F.col("category"), "(?i)Protien", "Protein")      # Fix in categories
    )
)

In [0]:
# Display sample records to verify spelling corrections applied successfully
display(df_silver.limit(5))

product_name,product_id,category,read_timestamp,file_name,file_size
SportsBar Energy Bar Choco Fudge (60g),25891101,Energy Bars,2025-11-30T13:40:08.692Z,products.csv,1388
SportsBar Energy Bar Choco Fudge (40g),25891102,Energy Bars,2025-11-30T13:40:08.692Z,products.csv,1388
SportsBar Energy Bar Choco Fudge (25g),25891103,Energy Bars,2025-11-30T13:40:08.692Z,products.csv,1388
SportsBar Protein Bar Peanut Crunch (45g),25891201,Protein Bars,2025-11-30T13:40:08.692Z,products.csv,1388
SportsBar Protein Bar Peanut Crunch (55g),25891202,Protein Bars,2025-11-30T13:40:08.692Z,products.csv,1388


#### QUALITY STEPS 4️⃣-7️⃣: Standardizing Product Attributes to Parent Company Data Model

**Objective:** Enrich product data to match parent company dimension structure

**Steps:**
- 4️⃣ Map raw categories to business divisions (category hierarchy)
- 5️⃣ Extract variant information from product names
- 6️⃣ Generate deterministic product_code for cross-system joins
- 7️⃣ Validate and standardize product_id with fallback handling

**Output:** Conformed product dimension ready for warehouse integration

In [0]:
### QUALITY STEP 4️⃣: Division Mapping - Map Categories to Business Divisions
# Business rule: Raw categories map to higher-level divisions for hierarchy
df_silver = (
    df_silver
    .withColumn(
        "division",
        F.when(F.col("category") == "Energy Bars",        "Nutrition Bars")
         .when(F.col("category") == "Protein Bars",       "Nutrition Bars")
         .when(F.col("category") == "Granola & Cereals",  "Breakfast Foods")
         .when(F.col("category") == "Recovery Dairy",     "Dairy & Recovery")
         .when(F.col("category") == "Healthy Snacks",     "Healthy Snacks")
         .when(F.col("category") == "Electrolyte Mix",    "Hydration & Electrolytes")
         .otherwise("Other")  # Catch-all for unmapped categories
    )
)

### QUALITY STEP 5️⃣: Extract Variant Information
# Extract text within parentheses as variant (e.g., "Product Name (Vanilla)" → "Vanilla")
# Regex pattern: \((.*?)\) captures content between parentheses
df_silver = df_silver.withColumn(
    "variant",
    F.regexp_extract(F.col("product_name"), r"\((.*?)\)", 1)  # Group 1 = content between parens
)

### QUALITY STEPS 6️⃣ & 7️⃣: Product Code Generation & ID Validation
# Invalid product_ids are replaced with a fallback value to avoid losing fact records 
# and ensure downstream joins remain consistent

df_silver = (
    df_silver
    # 1. Generate deterministic product_code from product_name via SHA256 hash
    #    Benefits: Same name always gets same code; consistent across systems
    .withColumn(
        "product_code",
        F.sha2(F.col("product_name").cast("string"), 256)  # 256-bit SHA2 hash
    )
    # 2. Clean product_id: keep only numeric IDs, else set to 999999
    #    Validation: regex pattern "^[0-9]+$" matches purely numeric strings
    #    Fallback: 999999 is reserved value for invalid/unknown product IDs
    .withColumn(
        "product_id",
        F.when(
            F.col("product_id").cast("string").rlike("^[0-9]+$"),  # Is numeric?
            F.col("product_id").cast("string")                      # Keep as string for join
        ).otherwise(F.lit(999999).cast("string"))                   # Invalid → 999999
    )
    # 3. Rename product_name → product for clarity (standardized naming convention)
    .withColumnRenamed("product_name", "product")
)

In [0]:
# Select final silver layer columns in logical order (key columns first, then attributes)
df_silver = df_silver.select(
    "product_code",      # Deterministic join key
    "division",          # Business division
    "category",          # Product category
    "product",           # Product name
    "variant",           # Product variant
    "product_id",        # Original product ID (validated)
    "read_timestamp",    # Data lineage - when processed
    "file_name",         # Data lineage - source file
    "file_size"          # Data lineage - data volume
)

In [0]:
# Display final silver layer data - verify all transformations applied correctly
display(df_silver)

product_code,division,category,product,variant,product_id,read_timestamp,file_name,file_size
e91ba9d665f90254da5809bfdebe3db2be01a52f50b6fd96b57eed238392b843,Nutrition Bars,Energy Bars,SportsBar Energy Bar Choco Fudge (60g),60g,25891101,2025-11-30T13:40:08.692Z,products.csv,1388
e92c739a8d78cd6cbe954648c2f9dd75ed61fcfd99b03e10dca65c3082d0728e,Nutrition Bars,Energy Bars,SportsBar Energy Bar Choco Fudge (40g),40g,25891102,2025-11-30T13:40:08.692Z,products.csv,1388
102628255d24304d6bbe0438b1ac992054f262e0814d306d0a34d7356cef3268,Nutrition Bars,Energy Bars,SportsBar Energy Bar Choco Fudge (25g),25g,25891103,2025-11-30T13:40:08.692Z,products.csv,1388
2e387cef1424d6e7b162b45622d4b1a788d11776e33d05cc8552f4ecd2ea1896,Nutrition Bars,Protein Bars,SportsBar Protein Bar Peanut Crunch (45g),45g,25891201,2025-11-30T13:40:08.692Z,products.csv,1388
0cb7b2f42657b625f754e833aa1cf6a967be26f17415f5342302ebb0e90c8a28,Nutrition Bars,Protein Bars,SportsBar Protein Bar Peanut Crunch (55g),55g,25891202,2025-11-30T13:40:08.692Z,products.csv,1388
889c67757ece9c973791dfbc2d47b026a3342cc7255e47a3170329d158e897c2,Nutrition Bars,Protein Bars,SportsBar Protein Bar Peanut Crunch (65g),65g,25891203,2025-11-30T13:40:08.692Z,products.csv,1388
3cab59f05924285270313afcfe40a08983bb03dd88f432e34fc6336914c14345,Breakfast Foods,Granola & Cereals,SportsBar Granola Crunch Honey Almond (400g),400g,25891301,2025-11-30T13:40:08.692Z,products.csv,1388
d9ebd1ca64d23951a6310af93b1c5ac27d831ac842e89aea59a9e8b38621faa5,Breakfast Foods,Granola & Cereals,SportsBar Granola Crunch Honey Almond (300g),300g,25891302,2025-11-30T13:40:08.692Z,products.csv,1388
c68834ceaff15846bc1892c2185dc4e4f471d64fe3796b1a8ecc39a5a48c614f,Breakfast Foods,Granola & Cereals,SportsBar Granola Crunch Honey Almond (200g),200g,25891303,2025-11-30T13:40:08.692Z,products.csv,1388
da6bfc596c1360ca07bda4e0ae6bfe3b8456517fc6e8ddc265630ff940f9ab05,Dairy & Recovery,Recovery Dairy,SportsBar Greek Yogurt Pro Vanilla (200g),200g,25891401,2025-11-30T13:40:08.692Z,products.csv,1388


In [0]:
# Write standardized data to silver layer with Change Data Feed for auditability
df_silver.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \  # Enable CDF for tracking transformations
 .option("mergeSchema", "true") \                 # Allow schema evolution if needed
 .mode("overwrite") \                             # Replace previous silver data
 .saveAsTable(f"{catalog}.{silver_schema}.{data_source}")

## STEP 5: GOLD LAYER - Business-Ready Product Dimension (Staging)

**Purpose:** Create SportsBar product dimension for fact table joins

**Medallion Pattern:** Select business-relevant columns only, drop technical metadata

**Key Columns:**
- `product_code` - Conformed dimension key (join with fact tables)
- `product_id` - Original source identifier
- `division` / `category` - Product hierarchy
- `product` / `variant` - Product details

**Output Table:** `fmcg.gold.sb_dim_products` (staging for parent merge)

In [0]:
# Read from silver layer - start with cleaned, standardized data
silver_query = f"SELECT * FROM {catalog}.{silver_schema}.{data_source};"
df_silver = spark.sql(silver_query)
# Select only business-relevant columns (drop technical metadata)
df_gold = df_silver.select("product_code", "product_id", "division", "category", "product", "variant")
df_gold.show(5)

+--------------------+----------+----------------+--------------+--------------------+-------+
|        product_code|product_id|        division|      category|             product|variant|
+--------------------+----------+----------------+--------------+--------------------+-------+
|2e387cef1424d6e7b...|  25891201|  Nutrition Bars|  Protein Bars|SportsBar Protein...|    45g|
|fe5a8036be4b9a787...|  25891402|Dairy & Recovery|Recovery Dairy|SportsBar Greek Y...|   120g|
|da6bfc596c1360ca0...|  25891401|Dairy & Recovery|Recovery Dairy|SportsBar Greek Y...|   200g|
|e91ba9d665f90254d...|  25891101|  Nutrition Bars|   Energy Bars|SportsBar Energy ...|    60g|
|0cb7b2f42657b625f...|  25891202|  Nutrition Bars|  Protein Bars|SportsBar Protein...|    55g|
+--------------------+----------+----------------+--------------+--------------------+-------+
only showing top 5 rows


In [0]:
# Write SportsBar product dimension (staging table before parent merge)
df_gold.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \  # Enable CDF for audit trail
 .mode("overwrite") \                             # Replace previous version
 .saveAsTable(f"{catalog}.{gold_schema}.sb_dim_{data_source}")

## STEP 6: MERGE with Parent Company Dimension

**Purpose:** Integrate SportsBar products into parent company's master product dimension

**Pattern:** Delta MERGE operation (upsert) for incremental updates

**Match Key:** `product_code` (deterministic hash ensures consistency)

**Operations:**
- **whenMatched:** Update existing parent records with latest SportsBar attributes
- **whenNotMatched:** Insert new products not yet in parent dimension

**Output Table:** `fmcg.gold.dim_products` (unified master data)

In [0]:
# Load parent company product dimension table
delta_table = DeltaTable.forName(spark, "fmcg.gold.dim_products")
# Read SportsBar products from gold staging table
df_child_products = spark.sql(f"SELECT product_code, division, category, product, variant FROM fmcg.gold.sb_dim_products;")
df_child_products.show(5)

+--------------------+----------------+--------------+--------------------+-------+
|        product_code|        division|      category|             product|variant|
+--------------------+----------------+--------------+--------------------+-------+
|2e387cef1424d6e7b...|  Nutrition Bars|  Protein Bars|SportsBar Protein...|    45g|
|fe5a8036be4b9a787...|Dairy & Recovery|Recovery Dairy|SportsBar Greek Y...|   120g|
|da6bfc596c1360ca0...|Dairy & Recovery|Recovery Dairy|SportsBar Greek Y...|   200g|
|e91ba9d665f90254d...|  Nutrition Bars|   Energy Bars|SportsBar Energy ...|    60g|
|0cb7b2f42657b625f...|  Nutrition Bars|  Protein Bars|SportsBar Protein...|    55g|
+--------------------+----------------+--------------+--------------------+-------+
only showing top 5 rows


In [0]:
# Perform MERGE operation to upsert SportsBar products into parent dimension
delta_table.alias("target").merge(
    source=df_child_products.alias("source"),
    # Match condition: Same product_code indicates matching product
    condition="target.product_code = source.product_code"
).whenMatchedUpdate(
    # Update existing records with latest SportsBar attributes
    set={
        "division": "source.division",
        "category": "source.category",
        "product": "source.product",
        "variant": "source.variant"
    }
).whenNotMatchedInsert(
    # Insert new products not found in parent dimension
    values={
        "product_code": "source.product_code",
        "division": "source.division",
        "category": "source.category",
        "product": "source.product",
        "variant": "source.variant"
    }
).execute()  # Execute the MERGE operation

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
# Read and display final merged product dimension to verify MERGE success
gold_query = f"SELECT * FROM {catalog}.{gold_schema}.dim_{data_source};"
df_gold = spark.sql(gold_query)
# Display merged parent dimension - should now include all SportsBar products
display(df_gold)

product_code,division,category,product,variant
ARCHDDE20D,Archery,Arrows,PX Carbon Arrow Set,12 Pack
ARCH158F41,Archery,Finger Tab,LX Leather Finger Tab,Universal
ARCHAFF0E4,Archery,Bow,AX Precision Recurve Bow,26 lbs
ARCH6B94F7,Archery,Bow,AX Precision Recurve Bow,30 lbs
ARCH5D1FE7,Archery,Bow Stringer,BX Bow Stringing Tool,Standard
ARCH7B49A9,Archery,Arm Guard,NX Archery Arm Guard,Medium
ARCH497D34,Archery,Target,TX Foam Archery Target,80 cm
ARCHE71D79,Archery,Arm Guard,NX Archery Arm Guard,Large
ARCHDD8749,Archery,Target,TX Foam Archery Target,60 cm
BADMC045D4,Badminton,Badminton Racket,BX AeroLite Badminton Racket,Head Heavy
